In [2]:
import sys, fitz, os, datetime
from PIL import Image
import pytesseract
import sys
import os
import re
import difflib
import numpy as np
import pandas as pd
import pdfplumber
#pd.set_option('display.max_rows', None)
sys.path.append('/opt/homebrew/Cellar/tesseract/5.3.3/bin/tesseract')

/Users/xuzhejia/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED

/Users/xuzhejia/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (



In [3]:
#各部長所代表的政黨
dict_name = {'朱主任敬一':'中國國民黨官員',
             '翁主任政義':'民主進步黨官員',
             '魏主任哲和':'民主進步黨官員',
             '吳主任茂昆':'民主進步黨官員',
             '陳主任建仁':'民主進步黨官員',
             '李主任羅權':'中國國民黨官員',
             '張部長善政':'中國國民黨官員',
             '林代理部長一平':'中國國民黨官員',
             '徐部長爵民':'中國國民黨官員',
             '楊部長弘敦':'民主進步黨官員',
             '陳部長良基':'民主進步黨官員',
             '吳部長政忠':'民主進步黨官員',
             '吳主任政忠':'民主進步黨官員'}

In [4]:
#純pdf
def handle_pdf(file_path, file_name):
    pdf = pdfplumber.open(f'../{file_path}/{file_name}/{file_name}.pdf')
    text = ''
    for x in range(len(pdf.pages)):
        text += pdf.pages[x].extract_text()
    with open(f'../{file_path}/{file_name}/{file_name}.txt', 'w') as f:
        f.write(text)

def find_target_pdf(file_path,file_name,屆數):
    df_party = pd.read_csv(f'../party_{屆數}.csv', index_col = 0)
    speaker = None
    speaker_list = []
    text_list = []
    with open(f'../{file_path}/{file_name}/{file_name}.txt', 'r') as f:
        text = f.readlines()
    name = df_party['name'].values
    for x in text:
        if ((x.split('：')[0].replace('委員','') in name) or ((re.search(r'(主席|委員|署長|次長|院長|主冷|處長|局長|主任|主持人|主廣|部長)', x) != None) and ('：' in x))) and '委員會' not in x:
            x = x.replace('\n','')
            speaker = x.split('：')[0].replace('委員', '')
            text = x.split('：')[1]
            speaker_list.append(speaker)
            text_list.append(text)
        elif (speaker != None) and ('立法院公報' not in x) and (re.match(r'^[0-9]*$',x) == None) and (len(x) != 0) and (re.match('\s',x) == None):
            text = x.replace('\n','')
            speaker_list.append(speaker)
            text_list.append(text)
    df = pd.DataFrame({'speaker':speaker_list, 'text': text_list})
    party_dict = dict(zip(df_party['name'], df_party['party']))
    for x in range(df.shape[0]):
        if (df.iloc[x, 0] in name): 
            df.loc[x, 'party'] = party_dict[df.iloc[x ,0]]
        elif (df.iloc[x, 0] in dict_name.keys()):
            df.loc[x, 'party'] = dict_name[df.iloc[x, 0]]
    df.to_csv(f'../{file_path}/{file_name}/one.csv')

In [7]:
# 處理圖片pdf
# 期中 吳茂昆、李羅權、翁政義、陳建仁、魏哲和
# 主席、委員、院長

#pdf轉png
def pdf_to_png(file_path):
    pdfDoc = fitz.open(file_path)
    for x in range(pdfDoc.page_count):
        page = pdfDoc[x]
        zoom = int(400)
        rotate = int(0)
        trans = fitz.Matrix(zoom / 100.0, zoom / 100.0).prerotate(rotate)
        pm = page.get_pixmap(matrix=trans, alpha=False)
        pm.writePNG(f'{file_path}_page{x}.png')

#png 轉 txt    
def png_to_string(file_path, file_name):
    all_file = os.listdir(file_path)
    all_file_filter = [x for x in all_file if '.png' in x]
    all_file_filter.sort(key = lambda x:int(re.split('.png|page', x)[1]))
    text = ''
    for file in all_file_filter[:80]:
        img = Image.open(file_path + '/' + file)
        text += pytesseract.image_to_string(img, lang = 'chi_tra').replace(' ','')
    with open(f'{file_path}/{file_name}.txt', 'w') as f:
        f.write(text)

#找出主席、委員、各級官員所講的話
def find_target(file_path, file_name):
    speaker = None
    content = []
    speaker_list = []
    text = open(f'{file_path}/{file_name}.txt', 'r')
    for text in text.readlines():
        print(text)
        # 如果有什麼新的職位想要扁別出來放這
        # 例如希望陳教授的發言被辨識可以將教授加入篩選條件
        if (len(text.split(':')) > 1) and (re.search(r'(主席|委員|署長|次長|院長|主冷|處長|局長|主任|主持人|主廣|主肺|主覓|主謹|委勛)',text.split(':')[0]) != None):
            speaker = text.split(':')[0]
            speaker_list.append(speaker)
            content.append(''.join(text.split(':')[1:]))
        elif speaker != None:
            speaker_list.append(speaker)
            content.append(text)
    df = pd.DataFrame({'speaker': speaker_list, 'text': content})
    df.to_csv(f'{file_path}/{file_name}.csv')
    return pd.DataFrame({'speaker': speaker_list, 'text': content})


#比對委員名字將錯誤名字修正
def similar_name(file_path, file_name, 屆數, *args):
    df_party = pd.read_csv(f'../party_{屆數}.csv', index_col = 0)
    df = pd.read_csv(f'{file_path}/{file_name}.csv', index_col=0)
    name = []
    ration = []
    for x in df['speaker']:
        if x in df_party['name']:
            name.append(x)
        else:
            temp = []
            list_party = df_party['name'].tolist()
            list_party.extend(args)
            for y in list_party:
                temp.append(difflib.SequenceMatcher(None, x, y).ratio())
            name.append(list_party[np.argmax(temp)])
            ration.append(np.max(temp))
    df['similar_name'] = name
    df['ration'] = ration
    for x in range(df.shape[0]):
        # 想要被做similary的條件
        # 當比對綠大於0.2且名字中存在委員主任部長主冷等字樣會去進行比對
        # 若名字中出現 主席(...)的pattern為了避免被取代成某個委員因此直接保留、副主任則是避免被比對成主任委員
        if (df.loc[x,'ration'] > 0.2) and (re.search(r'(委員|主任|部長|主席$|主冷|主廣|主覓|主肺|委勛|主謹)', df.loc[x,'speaker']) != None) and (re.search(r'中心吳主任|主席(.*)|副主任', df.loc[x, 'speaker']) == None):
            df.loc[x, 'speaker'] = name[x]
    df['party'] = df['speaker'].map(dict(zip(df_party['name'], df_party['party'])))
    df = df.loc[:,['speaker', 'text', 'party']]
    for x in range(df.shape[0]):
        if df.iloc[x, 0] in dict_name.keys():
            df.loc[x, 'party'] = dict_name[df.iloc[x, 0]]
    df.to_csv(f'{file_path}/{file_name}.csv')
    return df

def concat_text(file_path, file_name, file_type = 'picture'):
    df = pd.read_csv(f'{file_path}/{file_name}.csv', index_col=0)
    df['text'] = df['text'].astype(str)
    #在尾端加一列避免最後一個人被吃掉
    df.loc[df.shape[0]+1,:] = 'a'
    num_list = {}
    ans = []
    party = []
    name = []
    #利用num_list當作暫存器紀錄當下發言的人是誰弱下一句話依然同一人（利用是否存在暫存器當作判斷）發言就記錄起來
    #每次換人時暫存器需要更新
    for x in range(df.shape[0] - 1):
        if df.iloc[x,0] not in num_list:
            num_list[df.iloc[x,0]] = [df.iloc[x,1]]
        else:
            num_list[df.iloc[x,0]].append(df.iloc[x,1])
        if df.iloc[x + 1,0] != df.iloc[x, 0]:
            ans.append(''.join(num_list[df.iloc[x, 0]]))
            name.append(df.iloc[x,0])
            party.append(df.iloc[x,2])
            #ans.append(num_list[df.iloc[x, 0]])
            num_list = {}
    df = pd.DataFrame({'speaker':name, 'text':ans, 'party':party})
    df['text'] = df['text'].astype(str).str.replace('\n\n','').str.replace('\n','')

    df.to_csv(f'{file_path}/{file_name}.csv')
    return df

def get_consult(file_path, file_name, 屆數):
    df = pd.read_csv(f'{file_path}/{file_name}.csv', index_col= 0)
    df_party = pd.read_csv(f'../party_{屆數}.csv', index_col = 0)
    df['text'] = df['text'].astype(str)
    #利用主席說出請...答覆當作是分界（每個立委發言的分界）
    record = [x for x in range(df.shape[0]) if (re.search(r'(請.*答復|請.*說明)', df.iloc[x, 1]) != None) and (re.search('主席',df.iloc[x, 0]))]
    record.append(df.index[-1] + 2)
    print(record)
    conversation = [[record[x] - 1, record[x + 1]] for x in range(len(record) - 1)]
    #將主席（代理）的pattern替換掉
    speaker = [x.replace(x, '主席') if re.search(r'主席', x) != None else x for x in df['speaker'].astype(str)]
    df['speaker'] = speaker
    df_finish = pd.DataFrame([])
    #如果在member這個list中就會被無視（在移動party_last的過程中)
    member = df_party['name'].values.tolist()
    member.extend(list(dict_name.keys()))
    member.append('主席')
    for x in conversation:
        df_temp = df.loc[x[0]:x[1], :]
        #if set(df_temp['speaker'].unique()).issubset(set(member)):
        row_last = df_temp.index[0]
        for row in range(df_temp.index[0] + 1, df_temp.index[-1] + 1):
            if df_temp.loc[row, 'speaker'] in member and df_temp.loc[row,'speaker'] != '主席':
                #由上往下捕
                df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']
                row_last = row
        #補第一個party_last的空值
        if len(df_temp['party'].dropna().tolist()) > 1:
            df_temp.loc[df_temp.index[0], 'party_last'] = df_temp['party'].dropna().tolist()[1]
        df_temp = df_temp[:-2]
        df_finish = pd.concat([df_finish, df_temp])
    df_finish.to_csv(f'{file_path}/{file_name}_end.csv')


    
    

In [8]:
# 純pdf
name = '徐爵民'
files = [x for x in os.listdir('../徐爵民') if 'DS' not in x]
屆數 = '8'
for file in files:
    print(file)
    handle_pdf(name,file)
    find_target_pdf(name,file,屆數)
    concat_text(f'../{name}/{file}','one')
    get_consult(f'../{name}/{file}','one',屆數)

9_徐爵民_1040318_45620


/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/4149209195.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '中國國民黨官員' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[x, 'party'] = dict_name[df.iloc[x, 0]]



[4, 18, 28, 38, 41, 53, 73, 76, 111, 164, 214, 266, 288, 306, 337, 347, 389, 396, 401, 413, 444, 463, 473, 479, 482, 498, 560, 579, 626, 636, 645, 653, 662, 679, 688]
13_徐爵民_1050309_53195


/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '中國國民黨' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

[4, 44, 52, 93, 101, 104, 136, 170, 194, 203, 226, 264, 282, 285, 312, 328, 331, 367, 369, 385, 409, 433, 436, 449, 478, 528, 533, 564, 606, 629, 639, 649, 686, 708, 724, 755, 829, 885, 903, 918, 927, 931]
10_徐爵民_1040416_40216


...

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '中國國民黨' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

[0, 6, 9, 43, 48, 51, 56, 74, 79, 94, 97, 117, 147, 158, 169, 174, 176, 188, 193, 198, 201, 211, 239, 243, 253, 268, 270, 279, 281, 288, 303, 314, 317, 330, 340, 343, 351, 354, 360, 365, 387, 390, 395, 437, 440, 444]
15_徐爵民_1050504_43813


/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: FutureWarning: Setting an item of inco

[5, 31, 48, 56, 82, 123, 147, 153, 190, 195, 200, 207, 214, 221, 232, 250, 275, 324, 362, 382, 432, 437, 464, 499, 516, 576, 606, 631]


/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '民主進步黨' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_81711/3951479493.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [10]:
# 圖片
name = '陳建仁'
files = [x for x in os.listdir(f'../{name}') if '.DS_Store' not in x]
屆數 = '6'
for file in files:
    #pdf_to_png(f'../{name}/{file}/{file}.pdf')
    png_to_string(f'../{name}/{file}','one')
    find_target(f'../{name}/{file}','one')
    # similar_name 最後參數放想要比對出的結果
    similar_name(f'../{name}/{file}', 'one', f'{屆數}', '主席', '陳主任建仁')
    concat_text(f'../{name}/{file}','one')
    get_consult(f'../{name}/{file}','one',f'{屆數}')

立法院公報第95卷第43期委員會紀錄



立法院第6屆第4會期科技及資訊委員會第6次全體委員會議紀錄



時間中華民國95年10月19日(星期四)上午9時10分、下午2時31分



地“點[本院群賢樓802會議室



主。席莊委員和子



主席:出席委員已足法定人數,開會。進行報告事項。



報告事項



一、宣讀上次全體委員會議議事錄。



還



二、(上午9時至12時)邀請行政院公共工程委員會主任委員率同所屬單

、位主管作業務報告,,並備質詢。



三、(下午2時30分至5時30分)邀請行政院國家科學委員會主任委員率

南部科學工業園區管理局局長、行政院公共工程委員會主任委員、交通

部高速鐵路工程局局長就「南部科學工業園區高鐵減振工程現況」作專

案報告,並備質詢。



主席:現在進行上午議程。請行政院公共工程委員會吳主任委員報告。



吳主任委員澤成:



者、前言



主席、各位委員先進:



今天,有機會向各位委員先進報告本會業務,並聆聽指教,深感榮幸。



公共建設乃國家的基礎建設,是繁榮國家經濟,提升國民生活水準及促進產業升級的關鍵,

也是展現國家競爭力的重要指標。行政院公共工程委員會之設立,即在於統籌各項公共工程之

規劃、審議、協調與督導等事項,並且建構有效推動公共工程建設的制度,以期公共工程計畫

能在合理的預算經費內如期、如質完成。



為達成上述任務,面對全球快速變動與激烈競爭的新時代,公共建設的有效推動,對國家未

來的發展及人民生活品質的改善有極大影響。本會將以勇於創新、突破傳統思維之新觀念與作

法,為國家建設導入更多人性、優質、生態與活力,不論在政府採購效能的強化、營建產業的

發展、工程技術的提升、引進民間參與公共建設及各項重大建設進度與品質管控等各方面,均

統籌規劃推行,並深入瞭解問題所在,設法協調解決,以有效提升公共工程品質與執行續效,

打造人本、優質、永續的公共建設,為國家發展奠定厚實的基礎。在此謹就本會當前工作重點

與預期目標,從「法規制度」及「推動執行」二方面分別向各位委員先進報告。



憤、有關法規制度方面



一、建立優質廉能的採購環境



221

立法院公報第95卷第43期委員會紀錄



(一)建立公平、公正、公開之採購作業:規定招標、決標資訊公開於本會網站,

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '中國國民黨' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

立法院公報第96卷第51期委員會紀錄



立法院第6屆第5會期科技及資訊委員會第21次全體委員會議紀錄

時



才間中華民國96年6月4日(星期一)上午9時11分

地“點本院群賢樓802會議室

主席陳委員銀河

主席:出席委員已足法定人數,開會。



進行報告事項。

報告事項

一、宣讓上次全體委員會議議事錄。



主席:請問各位,上次會議議事錄有無錯誤?(無)無錯誤,確定。纖續報告。



、邀請行政院國家科學委員會主任委員及財團法人國家實驗研究院院長

率相關業務單位主管就「國家太空中心(NSPO)之ARGO太空計畫案

採購弊案」及「國家實驗研究院高速網路與計算中心中部科學工業園區

建築工程乙案」作專案報告,並備質詢。



主席:主委,現在還有些備詢官員沒有到,這樣不太好吧!



莊院長哲男:(在席位上)可能下雨塞車。



主席:委員會開會,9點就是9點,沒有什麼理由。由此可以看出國科會,尤其實驗研究院的管理

是全台灣最差的,真的需要檢討,哪有委員都到了,反而備詢官員沒有到?請問各位委員,有

沒有意見?



莊委員和子:(在席位上)可能是塞車。



主席:昨天下那麼大的雨,今天又是禮拜一,大家都知道會塞車,所以我7點就從桃園出發了。主

委,這種情況大家都看到了,哪有這樣搞的?我們暫時休息一下,等備詢官員到之後,再開會



休息



繼續開會



主席:現在繼續開會。馮委員定國要求就權宜問題發言,請滲委員發言。



馮委員定國:主席、各位同仁。我們國家的太空中心發生了這麼大的案子,相關人員不來列席報告

,這是置國家於何地?置人民於何地?行政院應該向立法院負責,現在立法院叫太空中心來這

裡向人民負責,太空中心代表不敢來,這像什麼話呢?太空中心是發射火箭上去,不是放沖天

炮,我們找的是放火箭的人,放火箭的人不來,放沖天炮的人來,做什麼用呢?



我們國家的太空中心,是政府花錢讓美國去做事,舉例來說,崗年的時候,有的有錢小孩

然買得起沖天炮,卻不敢放,有的小孩家裡窮,卻會放沖天炮,有錢小孩會把沖天炮拿給窮小

孩放,放炮的時候,出錢的小孩就把眼睛閉起來,手后住耳朵,等到放完了以後,再問放炮的

人放完了沒有。事情就是這樣子,我們當凱子,出錢叫人家做事,還被人「歪哥」,這還得了



_161

立法院公報第96卷第5

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '民主進步黨' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

立法院公報第96卷第53期委員會紀錄



立法院第6屆第5會期預算及決算委員會第6次全體委員會議紀錄



時間中華民國96年6月6日(星期三)上午10時36分



地”點[本院群賢樓502會議室



主“席廖召集委員本煙



主席:出席委員已足法定人數,開會。進行報告事項。



報告事項



一、宣讀上次會議議事錄。



主席:請問各位,上次會議議事錄有無錯誤?(無)無錯誤,確定。繼續報告。



二、請行政院國家科學委員會主任委員率同所屬主管人員列席報告

ARGO衛星計畫預算執行情形」並備詢及請行政院環境保護署代署長率

同所屬主管人員列席報告「推動環保材質製作餐具、紙杯等用具預算編

列及執行情形」並備詢。



主席:請國科會陳主任委員報告。



陳主任委員建仁:主席、各位委員。現由本會報告「ARGO衛星計畫預算執行情形」,由於太空

中心隸屬於財團法人國家實驗研究院,因此請太空中心的陳正興副主任報告。



主席:請國家太空中心陳副主任報告。



陳副主任正興:主席、各位委員。



太空科技的發展,是科技及國力展現重要指標。國家太空中心自籌設以來,建構台灣衛星發

展基礎,成功發射福爾摩沙衛星一號、二號及三號星系,完成福爾摩沙衛星系列衛星計畫,其太

空科學研究成果備受國際肯定。



謹就Argo衛星計畫相關預算執行情形報告如下:



一、原Argo計畫說明|



衛星系統發展計畫第一一和自主下展衛旦計畫主要目的是在建立我國衛星的傳承設計能量,及

自力發展世界第一組LEON-3CPU衛星電腦。



該計畫於確定以加入RapidEye星系計畫作為衛星任務後,將此枚衛星命名為ARGO衛星,

由國家太空中心主導研發角色,負責設計、組裝、整合測試、全部系統工程的工作,發展我國可

供傳承的衛星設計能量。由於是首枚國家太空中心自行設計的衛星,及加入國際衛星營運計畫的

驗證要求,該計畫另交選二組國外技術團隊,協助衛星設計與任務驗證。其中加拿大MDA公司

進行任務驗證工作,因無法取得加拿大政府輸出許可,國家太空中心經評估確認並徵詢律師意見

後,依程序報請國研院核准,於9年11月16日終止合約。



二、衛星系統發展第一枚自製衛星計畫之合約架構



衛星系統發展計畫之第一枚自製衛星,以建立完整的衛星本體自製能力及我國衛星本體的傳

...

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '民主進步黨' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

呈



法院公報“第95卷第39期委員會紀錄



立法院第6屆第4會期科技及資訊委員會第2次全體委員會會議紀錄



時”間民國95年10月2日(星期一)上午9時7分、下午14時38分



地“點[本院群賢樓802會議室



主席”李委員永萍



主席:出席委員已足法定人數,現在開會。進行報告事項。



報告事項



一、宣讀上次全體委員會議議事錄。



主席:請問各位,上次會議議事錄有無錯誤?請郭委員俊銘發言。



郭委員俊銘:主席、各位同仁。各委員會的召委都是由黨團協商各政黨分配幾個席次,政黨內部再

自行協商;到目前為凸,本黨團給我們的訊息是這部分的協商還未完成,所以議事錄的部分,可

否容許黨團繼續就委員的部分協商後,再來確認?



主席:向郭委員說明,在上次會議的時候,委員會已提供各位朝野協商各政黨所有黨容簽字的資料

,而且這部分也已經交付院會,包括選出來的召委也都已經核備,否則今天也沒有辦法在這裡台

開委員會。郭委員說這部分沒有完成,可能與事實沒有很相符合。郭委員如需要參考,這裡有朝

野協商的正本資料,及各政黨比鞭的親筆簽名。



郭委員俊銘:因為上星期五黨團給我們的意見就是這樣。所以本席覺得,如果黨團的協商沒有定案

的話,甚至連很重要的預算審議到了黨團協商都可以推勳。雖然主席個人能夠掌控今天的議事權

,但也請讓我們再次確認黨團的部分都沒有意見之後,再來確定議事錄。



主席:本席現在處理這件事情,是根據現有的書面資料,截至目前為止,我們這裡有立法院人事處

給各委員會的函,裡面說明已經有了朝野協商,及各委員會召委的席次,還有各政黨黨容的簽名

。我們是這根據相關的文件,才在上次會議將召委推舉出來。今天除非我們收到另外一份朝野協

商來推翻這份文件,否則本席無法根據郭委員的口述來做處理,何況也沒有其他人得到所屬黨團

黨讓不同的訊息,所以本席無法根據您的建議來做處理。



另外,立法院各委員會召集委員選舉辦法第5條已經寫得很清楚,即召集委員的選舉可以經

由各黨團及未參加黨團之該委員會委員之書面同意,以推選方式行之。所以,我們現在已經根據

相關的程序在進行。



這裡有一份民進黨簽署莊和子委員為科資委員會召委的推舉書,這些都已全部完成法定程序

,而且沒有任何書面証據表示民進黨團對這件事有意見,所以本席無法處理郭委

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '中國國民黨' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

立法院公報第96卷第12期委員會紀錄



立法院第6屆第4會期科技及資訊委員會第11次全體委員會會議紀錄



時間中華民國96年1月3日(星期三)上午9時26分

地“點本院群賢樓802會議室



主席李委員永薄



主席:出席委員已足法定人數,開會。進行報告事項。



報告事項

一、宣讀上次會議議事錄。

主席:請問各位,上次會議議事錄有無錯誤?(無)無錯誤,確定。

繼續報告。



二、邀請行政院政務委員林逢慶、行政院國家科學委員會主任委員陳建仁

率相關國家型計畫主持人、經濟部政務次長施顏祥率工業局局長陳昭義

等就「『數位典藏國家型計畫』、『數位學習國家型計畫』、【加強數

位內容產業發展推動方案』等計畫第一期推動成果、第二期推動方案等

事項。」作專案報告,並備質詢。



主席:本會審查相關國家型計畫預算時,與會委員非常關切「數位典藏國家型計畫」的加值運用,

也希望新計畫更具積極性,經深入了解後,我們知道數位型國家計畫未來無法單獨進行,必須與

「數位典藏國家型計畫」合併,如此一來,到底會使第一期的數位內容、發展策略、數位學習更

具成效還是減弱,本院委員都非常關心,今天特別邀請相關首長進行專案報告。

首先請行政院科技顧問組郭副執行秘書報告。



郭副執行秘書耀煌:主席、各位委員。今天大院第6屆第4會期「科技及資訊委員會」第11次

全體委員會議,承葵受邀列席報告,深感榮幸。以下謹代表行政院科技顧問組,就本組參與我國

數位內容相關政策之推動情形,提出報告如后,敬請各位委員指教。

壹、前言

台灣電子資訊業的熱競爭價值已從過去倚賴低附加價值、替代性高的生產與裝配活動,開始

推移至高附加價值,重視產品設計與創新的研究發展。數位內容產業具有發展知識經濟與數位經

濟之指標意義,除可促進傳統產業提升其知識含量而轉型成為知識型產業,亦是提升台灣整體產

業競爭力之基礎平台。



隨著網際網路普及數位匯流的趨勢,電信、網路、有線電視業者在既競爭又合作的生熊下,

融合成為一個更大的產業。數位匯流使消費大眾更容易取得數位內容,好的內容也因此扮演關鍵

性的角色,同時也創造了龐大商機與需求。



”貳、推動歷程

為扶植數位內容產業之發展,行政院於2002年5月13日核定通過「加強數位內容產業推動



145

立法院公報第96卷第12期

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '民主進步黨官員' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_temp.loc[row, 'party_last'] = df_temp.loc[row_last, 'party']

/var/folders/_0/d81h2sdj5z1b6t7hh803swth0000gn/T/ipykernel_87415/2347065050.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row